In [5]:
%pwd


'c:\\Users\\hp26\\Documents\\Text_Summarization_project'

In [3]:
import os

In [4]:
os.chdir("../")

entity file

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    rool_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps : int
    save_steps: int
    gradient_accumulation_steps: Path



configuration file

In [ ]:
from textSummarizer.constrants import *
from textSummarizer.utils.common import read_yaml,create_directories


class ConfigurationManager:
    def __init__(self, 
                config_filepath: CONFIG_FILE_PATH, 
                params_filepath: PARAMS_FILE_PATH ):
        self.config_filepath = read_yaml(config_filepath)
        self.params_filepath =read_yaml( params_filepath)

        create_directories( self.config.artifacts_root)


    def get_model_trainer_config(self)->ModelTrainerConfig:
            config = self.config.model_trainer
            params = self.params.TrainingArguments

            create_directories([config.root_dir])

            model_trainer_config = ModelTrainerConfig(
            rool_dir = config.artifacts_root,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation
            )

components file

In [12]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


In [ ]:
class ModelTrainer:
    def __init__ (self, config:ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else " cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        # loading data
        datasets_samsim_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir= self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay= self.config.weight_decay, 
            logging_steps = self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = self. 


        )

